In [1]:
import open3d.ml as _ml3d
import open3d as o3d
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [6]:
cfg_file = "configs/kpconv_parislille3d.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.KPFCNN(**cfg.model)
cfg.dataset['dataset_path'] = "/mnt/d/Krishna/Research/Datasets/Paris/"
dataset = ml3d.datasets.ParisLille3D(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

In [7]:
# download the weights.
ckpt_folder = "pretrained/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "kpconv_parislille3d_202011241550utc.pth"

In [8]:
# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

In [9]:
dataset.get_split_list("test")

['/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/000003.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/ajaccio_2.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/ajaccio_57.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/dijon_9.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/IIITB_base_Readable.ply']

In [10]:
test_split = dataset.get_split("test")
data = test_split.get_data(4)

In [11]:
# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

test 0/1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 443889/443889 [05:32<00:00, 57.39it/s]

In [12]:
len(result['predict_labels'])

2275320

In [13]:
set(result['predict_labels'])

{0, 1, 2, 5, 7, 8}

In [15]:
newData = data

In [16]:
newData['label'] = result['predict_labels']

In [17]:
newData

{'point': array([[-10.417  ,  -0.182  ,  -0.7285 ],
        [-10.302  ,  -0.23   ,  -0.779  ],
        [-10.325  ,  -0.23475,  -0.711  ],
        ...,
        [ 13.417  ,  10.72   ,  -0.692  ],
        [ 13.421  ,  10.711  ,  -0.685  ],
        [ 13.512  ,  11.25   ,  -0.745  ]], dtype=float32),
 'feat': None,
 'label': array([8, 8, 8, ..., 8, 8, 8])}

In [18]:
# evaluate performance on the test set; this will write logs to './logs'.
#pipeline.run_test()

## Raw labelling

In [19]:
import numpy as np

# Assuming newData is already defined as in your example
newData['label'] = newData['label'] + 1

# Combine all data into a single array for writing to a text file
# Structure: x, y, z, r, g, b, label
data_to_write = np.hstack((
    newData['point'],                       # x, y, z coordinates
    #newData['feat'][:, :3],                 # r, g, b colors
    newData['label'].reshape(-1, 1)         # label as a single column
))

# Save to a text file ckpt_00300
np.savetxt("ParisLilleOut.txt", data_to_write, fmt='%.6f %.6f %.6f %d')

print("Point cloud data with labels saved to output_point_cloud.txt")

Point cloud data with labels saved to output_point_cloud.txt


In [ ]:
import numpy as np

# Assuming newData is already defined as in your example

# Combine the point cloud data: x, y, z, r, g, b, label
# Ensure colors are normalized to [0, 1] range
colors = newData['feat'][:, :3] / 255.0
labels = newData['label'].reshape(-1, 1).astype(np.float32)
data_to_write = np.hstack((newData['point'], colors, labels))

# Create a header for the PCD file
header = f"""# .PCD v0.7 - Point Cloud Data file format
VERSION 0.7
FIELDS x y z rgb label
SIZE 4 4 4 4 4
TYPE F F F F F
COUNT 1 1 1 1 1
WIDTH {data_to_write.shape[0]}
HEIGHT 1
VIEWPOINT 0 0 0 1 0 0 0
POINTS {data_to_write.shape[0]}
DATA ascii
"""

# Convert RGB values from float (0-1) to packed float (for PCD)
rgb_packed = np.array(
    ((colors[:, 0] * 255).astype(np.uint32) << 16) |
    ((colors[:, 1] * 255).astype(np.uint32) << 8) |
    ((colors[:, 2] * 255).astype(np.uint32))
).view(np.float32).reshape(-1, 1)

# Combine points, packed RGB, and labels
pcd_data = np.hstack((newData['point'], rgb_packed, labels))

# Write to the PCD file
with open("output_point_cloud_06500", "w") as f:
    f.write(header)
    np.savetxt(f, pcd_data, fmt="%.6f %.6f %.6f %.6f %.1f")

print("Point cloud with labels saved to output_with_labels.pcd")


test 0/1: 100%|███████████████████████████████████████████████████████████████| 692214/692214 [00:54<00:00, 2143.62it/s]